In [229]:
from sklearn.preprocessing import MinMaxScaler
from nltk.corpus import stopwords
english_stopwords = stopwords.words('english')
stopwords = set(english_stopwords)
import json
import pandas as pd
from tqdm import tqdm

In [230]:
import nltk
from nltk.corpus import stopwords
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 50))
# read card_deck from ppn_deck.json file
with open("ppn_deck.json", "r") as read_file:
    card_deck = json.load(read_file)

def revise_point_values(card_deck, max_points=50):
    # Initialize a dictionary to store the word counts for each card
    word_counts = {}
    # Initialize a list to store the word counts for each card
    card_word_counts = []
    related_words = []
    # Get a list of English stopwords
    stop_words = stopwords.words("english")
    
    # Iterate through each card in the card deck
    card_id = 0
    for card in tqdm(card_deck):
        # Get the card's id from the iteration
        # Get the summary for the card
        summary = card['summary']
        # Split the summary into words. Currently it is a tuple, get the second element.
        summary_text = summary[1]
        # Split the summary into words
        words = summary_text.split()
        # Remove the stop words from the list of words
        nonstop_words = [word for word in words if word not in stop_words]
        # Calculate the number of non-stop words
        word_count = len(nonstop_words)
        # Add the word count to the dictionary
        word_counts[card_id] = word_count
        # Append the word count to the list of card word counts
        card_word_counts.append(word_count)
        # append related words to card deck
        try:
            related_words.append(int(card['related']))
        except:
            related_words.append(0)

        # Increment the card id
        card_id += 1
    # Scale the word counts to be between 0 and 10
    scaled_word_counts = scaler.fit_transform(pd.DataFrame(card_word_counts))
    # Iterate through each card in the card deck
    card_id = 0
    for card in tqdm(card_deck):
        #print(card)
        # Get the card's id from the iteration
        # Get the scaled word count for the card
        scaled_word_count = scaled_word_counts[card_id]
        # Get the card's point value if it exists
        # point_value = card['point_value'] if 'point_value' in card else 0
        related_words = int(card['related'])
        
        rounded_value = int(scaled_word_count[0])

        # Update the card's point value to the scaled word count
        try:
            card['point_value'] = max(rounded_value, related_words)
            print(card['point_value'])
        except Exception as e:
            print(e)
            print(card['point_value'])
            print(rounded_value)
            print(related_words)
            print(card)
        # Increment the card id
        card_id += 1
    return card_deck
    


To fix this, you can reshape card_word_counts to a 2D array using the reshape method of the numpy library, like this: card_word_counts.reshape(-1, 1). This will transform card_word_counts into a 2D array with a single column and as many rows as there are elements in card_word_counts.

You can then pass the reshaped array to the fit method of the MinMaxScaler object.

You will also need to make the same change to the line card_word_counts_scaled = scaler.fit_transform(card_word_counts.reshape(-1, 1)) in order to avoid the same error when you try to fit the scaler to the data.

Finally, in the for loop where you update the point value for each card, you should use card_word_counts_scaled[card_id] instead of card_word_counts_scaled[word_count]. This is because card_word_counts_scaled is a list of scaled word counts, and you need to use the current card_id to index into the list

In [231]:
# Now assign points based on the related column but scale it with MinMaxScaler
# card_deck = revise_point_values(card_deck, max_points=50)
scaler = MinMaxScaler(feature_range=(0, 50))
# read card_deck from ppn_deck.json file
with open("ppn_deck.json", "r") as read_file:
    card_deck = json.load(read_file)

def revise_point_values(card_deck, max_points=50):
    # Initialize a dictionary to store the word counts for each card
    word_counts = {}
    # Initialize a list to store the word counts for each card
    card_word_counts = []
    related_words = []
    # Get a list of English stopwords
    stop_words = stopwords.words("english")
    
    # Iterate through each card in the card deck
    card_id = 0
    for card in tqdm(card_deck):
        # Get the card's id from the iteration
        # Get the summary for the card
        summary = card['summary']
        # Split the summary into words. Currently it is a tuple, get the second element.
        summary_text = summary[1]
        # Split the summary into words
        words = summary_text.split()
        # Remove the stop words from the list of words
        nonstop_words = [word for word in words if word not in stop_words]
        # Calculate the number of non-stop words
        word_count = len(nonstop_words)
        # Add the word count to the dictionary
        word_counts[card_id] = word_count
        # Append the word count to the list of card word counts
        card_word_counts.append(word_count)
        # append related words to card deck
        try:
            related_words.append(int(card['related']))
        except:
            related_words.append(0)

        # Increment the card id
        card_id += 1
    # Scale the word counts to be between 0 and 10
    scaled_word_counts = scaler.fit_transform(pd.DataFrame(related_words))
    # Iterate through each card in the card deck
    card_id = 0
    for card in tqdm(card_deck):
        #print(card)
        # Get the card's id from the iteration
        # Get the scaled word count for the card
        try:
            scaled_word_count = scaled_word_counts[card_id]
            # Get the card's point value if it exists
            # point_value = card['point_value'] if 'point_value' in card else 0
            related_words = int(card['related'])
            
            rounded_value = int(scaled_word_count[0])
        except:
            related_words = 0
            rounded_value = 0
        # Update the card's point value to the scaled word count
        try:
            card['point_value'] = max(rounded_value, related_words)
            print(card['point_value'])
        except Exception as e:
            print(e)
            print(card['point_value'])
            print(rounded_value)
            print(related_words)
            print(card)
        # Increment the card id
        card_id += 1
    return card_deck

# Now assign points based on the related column but scale it with MinMaxScaler
card_deck = revise_point_values(card_deck, max_points=50)

100%|██████████| 6599/6599 [00:00<00:00, 321014.74it/s]

401
321
164
591
271
773
0
334
60
34
540
132
57
27
72
147
180
257
38
162
71
194
1337
67
910
23
114
329
56
33
259
412
462
131
71
86
2354
643
216
786
239
20
34
699
43
45
16
102
334
93
1681
959
128
103
240
80
171
13
463
108
50
210
356
1
788
730
110
228
48
117
66
786
17
16
342
36
608
2
24
291
359
32
373
62
43
35
88
669
332
203
34
115
875
367
53
25
96
241
70
94
47
34
520
42
148
64
228
276
17
194
49
165
40
184
778
475
342
354
59
45
68
261
82
31
147
18
1170
251
988
42
249
66
318
331
38
144
304
69
741
10
1569
132
125
496
72
602
109
572
157
423
160
51
198
352
220
22
365
46
620
30
14
1022
174
723
337
106
550
1041
351
1039
160
157
86
133
67
214
620
107
128
499
596
561
34
70
14
157
34
111
104
12
331
458
123
19
64
184
42
36
43
36
12
156
184
59
26
1825
2026
131
583
212
987
90
41
34
44
244
112
1989
275
24
211
109
125
326
82
2023
94
152
23
39
14
1498
24
103
55
1405
87
55
6
56
340
152
98
323
180
1
271
283
259
278
63
793
19
296
16
78
160
4
19
62
203
77
738
43
919
260
334
36
94
304
10
1857
85
19
426
40
91

In [232]:

# show an example with the revised point values
# convert to dataframe
df = pd.DataFrame(card_deck)
# show the first 5 rows
df.head()

title  \
0      Climate change and cities   
1              Rosetta Cammeniti   
2  Spy Muppets: License to Croak   
3       Fame in the 20th Century   
4                   Becca Hayton   

                                             summary related  \
0  Climate change and cities are deeply connected...     401   
1  Rosetta Cammeniti, Rosetta Rosie Cammeniti is ...     321   
2  Spy Muppets: License to Croak, Spy Muppets: Li...     164   
3  Fame in the 20th Century, Fame in the 20th Cen...     591   
4  Becca Hayton, Becca Dean also Hayton is a fict...     271   

                                       summary_short  \
0  Climate change and cities are deeply connected...   
1  Rosetta Cammeniti, Rosetta Rosie Cammeniti is ...   
2  Spy Muppets: License to Croak, Spy Muppets: Li...   
3  Fame in the 20th Century, Fame in the 20th Cen...   
4  Becca Hayton, Becca Dean also Hayton is a fict...   

                                       summary_clean  point_value  
0  Moreover because of processes that create clim...          401  
1                                                NaN          321  
2                                                NaN          164  
3                                                NaN          591  
4                                                NaN          271

In [233]:
# the summary_short needs to always be the second element in the tuple if it is an instance of a tuple, else it is itself (str)
for card in card_deck:
    summary = card['summary']
    if isinstance(summary, tuple) or isinstance(summary, list):
        summary_short = summary[1]
    else:
        summary_short = summary
    card['summary_short'] = summary_short
    card['summary'] = summary_short

df = pd.DataFrame(card_deck)

In [234]:
# revise point values
card_deck = revise_point_values(card_deck,20)


100%|██████████| 6599/6599 [00:00<00:00, 358405.36it/s]

401
321
164
591
271
773
0
334
60
34
540
132
57
27
72
147
180
257
38
162
71
194
1337
67
910
23
114
329
56
33
259
412
462
131
71
86
2354
643
216
786
239
20
34
699
43
45
16
102
334
93
1681
959
128
103
240
80
171
13
463
108
50
210
356
1
788
730
110
228
48
117
66
786
17
16
342
36
608
2
24
291
359
32
373
62
43
35
88
669
332
203
34
115
875
367
53
25
96
241
70
94
47
34
520
42
148
64
228
276
17
194
49
165
40
184
778
475
342
354
59
45
68
261
82
31
147
18
1170
251
988
42
249
66
318
331
38
144
304
69
741
10
1569
132
125
496
72
602
109
572
157
423
160
51
198
352
220
22
365
46
620
30
14
1022
174
723
337
106
550
1041
351
1039
160
157
86
133
67
214
620
107
128
499
596
561
34
70
14
157
34
111
104
12
331
458
123
19
64
184
42
36
43
36
12
156
184
59
26
1825
2026
131
583
212
987
90
41
34
44
244
112
1989
275
24
211
109
125
326
82
2023
94
152
23
39
14
1498
24
103
55
1405
87
55
6
56
340
152
98
323
180
1
271
283
259
278
63
793
19
296
16
78
160
4
19
62
203
77
738
43
919
260
334
36
94
304
10
1857
85
19
426
40
91

In [235]:
# the point_value must always be a value from 1 to 10 never 0. Replace all 0's with 1's
df['point_value'] = df['point_value'].replace(0,1)


In [236]:
df.head()

title  \
0      Climate change and cities   
1              Rosetta Cammeniti   
2  Spy Muppets: License to Croak   
3       Fame in the 20th Century   
4                   Becca Hayton   

                                             summary related  \
0  Climate change and cities are deeply connected...     401   
1  Rosetta Cammeniti, Rosetta Rosie Cammeniti is ...     321   
2  Spy Muppets: License to Croak, Spy Muppets: Li...     164   
3  Fame in the 20th Century, Fame in the 20th Cen...     591   
4  Becca Hayton, Becca Dean also Hayton is a fict...     271   

                                       summary_short  \
0  Climate change and cities are deeply connected...   
1  Rosetta Cammeniti, Rosetta Rosie Cammeniti is ...   
2  Spy Muppets: License to Croak, Spy Muppets: Li...   
3  Fame in the 20th Century, Fame in the 20th Cen...   
4  Becca Hayton, Becca Dean also Hayton is a fict...   

                                       summary_clean  point_value  
0  Moreover because of processes that create clim...          401  
1                                                NaN          321  
2                                                NaN          164  
3                                                NaN          591  
4                                                NaN          271

In [237]:
# plot a histogram of the point values
df['point_value'].hist()

<AxesSubplot: >

In [238]:
# now save the df as a json file
card_deck = df.to_dict('records')
with open("ppn_deck.json", "w") as write_file:
    json.dump(card_deck, write_file)
